In [12]:
import yfinance as yf
import numpy as np
import pandas as pd
from scipy.optimize import minimize

# 1. 자산 설정
tickers = ['SPY', 'QQQ', 'TLT', 'GLD', 'SCHD']
start_date = '2018-01-01'
end_date = '2024-12-31'

# 2. 데이터 다운로드 (auto_adjust=True가 기본이므로 Close 사용)
data = yf.download(tickers, start=start_date, end=end_date)['Close']
returns = data.pct_change().dropna()

# 3. 연간 수익률과 공분산
mean_returns = returns.mean() * 252
cov_matrix = returns.cov() * 252
rf_rate = 0.02  # 무위험 수익률 (연 2%)

# 4. 포트폴리오 성과 계산 함수
def portfolio_performance(weights):
    port_return = np.dot(weights, mean_returns)
    port_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe_ratio = (port_return - rf_rate) / port_volatility
    return port_return, port_volatility, sharpe_ratio

# 5. 목적 함수 (Sharpe Ratio를 -로 반환해 최소화)
def neg_sharpe_ratio(weights):
    return -portfolio_performance(weights)[2]

# 6. 제약 조건: 합은 1, 변동성 ≤ 10%
def volatility_constraint(weights):
    return 0.10 - portfolio_performance(weights)[1]

constraints = (
    {'type': 'eq', 'fun': lambda x: np.sum(x) - 1},              # 비중의 합 = 1
    {'type': 'ineq', 'fun': volatility_constraint}               # 변동성 ≤ 10%
)

bounds = tuple((0, 1) for _ in range(len(tickers)))  # 각 자산 비중 [0, 1] 범위
initial_weights = np.array([1/len(tickers)] * len(tickers))     # 균등 초기값

# 7. 최적화 수행
result = minimize(neg_sharpe_ratio, initial_weights, method='SLSQP',
                  bounds=bounds, constraints=constraints)

# 8. 결과 출력
opt_weights = result.x
opt_return, opt_volatility, opt_sharpe = portfolio_performance(opt_weights)

print("\n✅ 안정형 최적 포트폴리오 (변동성 ≤ 10%)")
for ticker, weight in zip(tickers, opt_weights):
    print(f"{ticker}: {weight:.2%}")
print(f"\n- 연 수익률: {opt_return:.2%}")
print(f"- 연 변동성: {opt_volatility:.2%}")
print(f"- Sharpe Ratio: {opt_sharpe:.2f}")

[*********************100%***********************]  5 of 5 completed


✅ 안정형 최적 포트폴리오 (변동성 ≤ 10%)
SPY: 39.19%
QQQ: 0.90%
TLT: 19.38%
GLD: 14.62%
SCHD: 25.92%

- 연 수익률: 8.13%
- 연 변동성: 10.00%
- Sharpe Ratio: 0.61


✅ 안정형 최적 포트폴리오 (변동성 ≤ 10%) <br>
SPY: 39.19% <br>
QQQ: 0.90% <br>
TLT: 19.38% <br>
GLD: 14.62% <br>
SCHD: 25.92% <br>
 <br>
- 연 수익률: 8.13% <br>
- 연 변동성: 10.00% <br>
- Sharpe Ratio: 0.61 <br>

In [13]:
import yfinance as yf
import numpy as np
import pandas as pd
from scipy.optimize import minimize

# 1. 자산 설정
tickers = ['SPY', 'TLT', 'GLD', 'SCHD']
start_date = '2018-01-01'
end_date = '2024-12-31'

# 2. 데이터 다운로드 (auto_adjust=True가 기본이므로 Close 사용)
data = yf.download(tickers, start=start_date, end=end_date)['Close']
returns = data.pct_change().dropna()

# 3. 연간 수익률과 공분산
mean_returns = returns.mean() * 252
cov_matrix = returns.cov() * 252
rf_rate = 0.02  # 무위험 수익률 (연 2%)

# 4. 포트폴리오 성과 계산 함수
def portfolio_performance(weights):
    port_return = np.dot(weights, mean_returns)
    port_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe_ratio = (port_return - rf_rate) / port_volatility
    return port_return, port_volatility, sharpe_ratio

# 5. 목적 함수 (Sharpe Ratio를 -로 반환해 최소화)
def neg_sharpe_ratio(weights):
    return -portfolio_performance(weights)[2]

# 6. 제약 조건: 합은 1, 변동성 ≤ 10%
def volatility_constraint(weights):
    return 0.10 - portfolio_performance(weights)[1]

constraints = (
    {'type': 'eq', 'fun': lambda x: np.sum(x) - 1},              # 비중의 합 = 1
    {'type': 'ineq', 'fun': volatility_constraint}               # 변동성 ≤ 10%
)

bounds = tuple((0, 1) for _ in range(len(tickers)))  # 각 자산 비중 [0, 1] 범위
initial_weights = np.array([1/len(tickers)] * len(tickers))     # 균등 초기값

# 7. 최적화 수행
result = minimize(neg_sharpe_ratio, initial_weights, method='SLSQP',
                  bounds=bounds, constraints=constraints)

# 8. 결과 출력
opt_weights = result.x
opt_return, opt_volatility, opt_sharpe = portfolio_performance(opt_weights)

print("\n✅ 안정형 최적 포트폴리오 (변동성 ≤ 10%)")
for ticker, weight in zip(tickers, opt_weights):
    print(f"{ticker}: {weight:.2%}")
print(f"\n- 연 수익률: {opt_return:.2%}")
print(f"- 연 변동성: {opt_volatility:.2%}")
print(f"- Sharpe Ratio: {opt_sharpe:.2f}")

[*********************100%***********************]  4 of 4 completed


✅ 안정형 최적 포트폴리오 (변동성 ≤ 10%)
SPY: 39.26%
TLT: 18.54%
GLD: 16.39%
SCHD: 25.80%

- 연 수익률: 8.13%
- 연 변동성: 10.00%
- Sharpe Ratio: 0.61


✅ 안정형 최적 포트폴리오 (변동성 ≤ 10%) <BR>
SPY: 39.26% <BR>
TLT: 18.54% <BR>
GLD: 16.39% <BR>
SCHD: 25.80% <BR>
 <BR>
- 연 수익률: 8.13% <BR>
- 연 변동성: 10.00% <BR>
- Sharpe Ratio: 0.61 <BR>

In [19]:
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.optimize import minimize

# 1. 자산 설정 및 데이터 로딩
tickers = ['SPY', 'QQQ', 'SCHD', 'TLT', 'GLD']
start_date = '2018-01-01'
end_date = '2024-12-31'

data = yf.download(tickers, start=start_date, end=end_date)['Close']
returns = data.pct_change().dropna()

# 2. 연 수익률 및 공분산 계산
mean_returns = returns.mean() * 252
cov_matrix = returns.cov() * 252
rf = 0.02

# 3. Sharpe Ratio (또는 Sortino) 최적화 함수
def neg_sharpe_ratio(weights, mean_returns, cov_matrix, rf):
    portfolio_return = np.dot(weights, mean_returns)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe_ratio = (portfolio_return - rf) / portfolio_volatility
    return -sharpe_ratio

# 4. 제약 조건 추가: 수익률, 변동성 범위 포함
def get_return(weights):
    return np.dot(weights, mean_returns)

def get_volatility(weights):
    return np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

constraints = [
    {'type': 'eq', 'fun': lambda x: np.sum(x) - 1},  # 합계는 100%
    {'type': 'ineq', 'fun': lambda x: get_return(x) - 0.04},  # 연 수익률 ≥ 4%
    {'type': 'ineq', 'fun': lambda x: 0.07 - get_return(x)},  # 연 수익률 ≤ 7%
    {'type': 'ineq', 'fun': lambda x: get_volatility(x) - 0.05},  # 변동성 ≥ 5%
    {'type': 'ineq', 'fun': lambda x: 0.10 - get_volatility(x)}   # 변동성 ≤ 10%
]

bounds = tuple((0, 1) for _ in range(len(tickers)))
initial_weights = [1 / len(tickers)] * len(tickers)

optimized = minimize(neg_sharpe_ratio, initial_weights,
                     args=(mean_returns, cov_matrix, rf),
                     method='SLSQP', bounds=bounds, constraints=constraints)

# 5. 결과 출력
if optimized.success:
    opt_weights = optimized.x
    opt_return = get_return(opt_weights)
    opt_volatility = get_volatility(opt_weights)
    opt_sharpe = (opt_return - rf) / opt_volatility

    print("🔍 최적화된 비중 (안정형 포트폴리오):")
    for ticker, weight in zip(tickers, opt_weights):
        print(f"{ticker}: {weight:.2%}")

    print(f"\n📈 예상 연 수익률: {opt_return:.2%}")
    print(f"📉 예상 연 변동성: {opt_volatility:.2%}")
    print(f"⭐ Sharpe Ratio: {opt_sharpe:.2f}")
else:
    print("❌ 최적화 실패: 제약 조건을 만족하는 해를 찾지 못했습니다.")

[*********************100%***********************]  5 of 5 completed

🔍 최적화된 비중 (안정형 포트폴리오):
SPY: 34.45%
QQQ: 0.00%
SCHD: 27.29%
TLT: 6.41%
GLD: 31.84%

📈 예상 연 수익률: 7.00%
📉 예상 연 변동성: 9.75%
⭐ Sharpe Ratio: 0.51


🔍 최적화된 비중 (안정형 포트폴리오): <br>
SPY: 34.45% <br>
QQQ: 0.00% <br>
SCHD: 27.29% <br>
TLT: 6.41% <br>
GLD: 31.84% <br>
 <br>
📈 예상 연 수익률: 7.00% <br>
📉 예상 연 변동성: 9.75% <br>
⭐ Sharpe Ratio: 0.51 <br>

In [20]:
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.optimize import minimize
import matplotlib.pyplot as plt

# 1. 자산 설정 및 데이터 로딩
tickers = ['SPY', 'QQQ', 'SCHD', 'TLT', 'GLD']
start_date = '2018-01-01'
end_date = '2024-12-31'

# 수정된 데이터 다운로드 부분
data = yf.download(tickers, start=start_date, end=end_date)  # 기본 auto_adjust=True
data = data['Close']  # 'Adj Close' 대신 'Close' 열 선택
returns = data.pct_change().dropna()

# 2. 연간 수익률 및 공분산 계산
mean_returns = returns.mean() * 252
cov_matrix = returns.cov() * 252
rf = 0.02  # 무위험 수익률 (2% 가정)

# 3. Sharpe Ratio를 음수로 변환 (최소화 함수이므로)
def neg_sharpe_ratio(weights, mean_returns, cov_matrix, rf):
    portfolio_return = np.dot(weights, mean_returns)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe_ratio = (portfolio_return - rf) / portfolio_volatility
    return -sharpe_ratio

# 4. 제약 조건 및 경계 설정 (변동성 제약 포함)
constraints = (
    {'type': 'eq', 'fun': lambda x: np.sum(x) - 1},  # 비중 합 100%
    {'type': 'ineq', 'fun': lambda x: np.dot(x, mean_returns) - 0.04},  # 수익률 ≥ 4%
    {'type': 'ineq', 'fun': lambda x: 0.07 - np.dot(x, mean_returns)},  # 수익률 ≤ 7%
    {'type': 'ineq', 'fun': lambda x: np.sqrt(np.dot(x.T, np.dot(cov_matrix, x))) - 0.05},  # 변동성 ≥ 5%
    {'type': 'ineq', 'fun': lambda x: 0.10 - np.sqrt(np.dot(x.T, np.dot(cov_matrix, x)))}   # 변동성 ≤ 10%
)

bounds = tuple((0, 1) for _ in range(len(tickers)))             # 0~1 사이

# 5. 초기 guess 및 최적화
initial_weights = [0.25] * len(tickers)
optimized = minimize(neg_sharpe_ratio, initial_weights,
                     args=(mean_returns, cov_matrix, rf),
                     method='SLSQP', bounds=bounds, constraints=constraints)

# 6. 결과 출력
opt_weights = optimized.x
opt_return = np.dot(opt_weights, mean_returns)
opt_volatility = np.sqrt(np.dot(opt_weights.T, np.dot(cov_matrix, opt_weights)))
opt_sharpe = (opt_return - rf) / opt_volatility

print("🔍 Sharpe 최대화 포트폴리오 (수익률 4~7%, 변동성 5~10% 제약):")
for ticker, weight in zip(tickers, opt_weights):
    print(f"{ticker}: {weight:.2%}")

print(f"\n📈 예상 연 수익률: {opt_return:.2%}")
print(f"📉 예상 연 변동성: {opt_volatility:.2%}")
print(f"⭐ Sharpe Ratio: {opt_sharpe:.2f}")

[*********************100%***********************]  5 of 5 completed


🔍 Sharpe 최대화 포트폴리오 (수익률 4~7%, 변동성 5~10% 제약):
SPY: 34.43%
QQQ: 0.00%
SCHD: 27.32%
TLT: 6.40%
GLD: 31.84%

📈 예상 연 수익률: 7.00%
📉 예상 연 변동성: 9.75%
⭐ Sharpe Ratio: 0.51


In [16]:
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.optimize import minimize
import matplotlib.pyplot as plt

# 1. 자산 설정 및 데이터 로딩
tickers = ['SPY', 'QQQ', 'SCHD', 'TLT', 'GLD']
start_date = '2018-01-01'
end_date = '2024-12-31'

# 수정된 데이터 다운로드 부분
data = yf.download(tickers, start=start_date, end=end_date)  # 기본 auto_adjust=True
data = data['Close']  # 'Adj Close' 대신 'Close' 열 선택
returns = data.pct_change().dropna()

# 2. 연간 수익률 및 공분산 계산
mean_returns = returns.mean() * 252
cov_matrix = returns.cov() * 252
rf = 0.02  # 무위험 수익률 (2% 가정)

# 3. Sharpe Ratio를 음수로 변환 (최소화 함수이므로)
def neg_sharpe_ratio(weights, mean_returns, cov_matrix, rf):
    portfolio_return = np.dot(weights, mean_returns)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe_ratio = (portfolio_return - rf) / portfolio_volatility
    return -sharpe_ratio

# 4. 제약 조건 및 경계 설정
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})  # 비중 합 100%
bounds = tuple((0, 1) for _ in range(len(tickers)))             # 0~1 사이

# 5. 초기 guess 및 최적화
initial_weights = [0.25] * len(tickers)
optimized = minimize(neg_sharpe_ratio, initial_weights,
                     args=(mean_returns, cov_matrix, rf),
                     method='SLSQP', bounds=bounds, constraints=constraints)

# 6. 결과 출력
opt_weights = optimized.x
opt_return = np.dot(opt_weights, mean_returns)
opt_volatility = np.sqrt(np.dot(opt_weights.T, np.dot(cov_matrix, opt_weights)))
opt_sharpe = (opt_return - rf) / opt_volatility

print("🔍 최적화된 비중 (Sharpe 최대) (변동성 제약 없음):")
for ticker, weight in zip(tickers, opt_weights):
    print(f"{ticker}: {weight:.2%}")

print(f"\n📈 예상 연 수익률: {opt_return:.2%}")
print(f"📉 예상 연 변동성: {opt_volatility:.2%}")
print(f"⭐ Sharpe Ratio: {opt_sharpe:.2f}")

[*********************100%***********************]  5 of 5 completed

🔍 최적화된 비중 (Sharpe 최대) (변동성 제약 없음):
SPY: 54.46%
QQQ: 45.54%
SCHD: 0.00%
TLT: 0.00%
GLD: 0.00%

📈 예상 연 수익률: 15.00%
📉 예상 연 변동성: 14.21%
⭐ Sharpe Ratio: 0.91


🔍 최적화된 비중 (Sharpe 최대) (변동성 제약 없음): <BR>
SPY: 54.46% <BR>
QQQ: 45.54% <BR>
SCHD: 0.00% <BR>
TLT: 0.00% <BR>
GLD: 0.00% <BR>
 <BR>
📈 예상 연 수익률: 15.00% <BR>
📉 예상 연 변동성: 14.21% <BR>
⭐ Sharpe Ratio: 0.91 <BR>

- 공격형으로 사용

In [21]:
import numpy as np
import pandas as pd
import yfinance as yf

# 1. 자산 설정 및 데이터 로딩
tickers = ['SPY', 'QQQ', 'SCHD', 'TLT', 'GLD']
start_date = '2018-01-01'
end_date = '2024-12-31'

data = yf.download(tickers, start=start_date, end=end_date)['Close']
returns = data.pct_change().dropna()

# 2. 연간 수익률 및 공분산 계산
mean_returns = returns.mean() * 252
cov_matrix = returns.cov() * 252
rf = 0.02  # 무위험 수익률 2%

# 3. 주어진 비중 배열로 변환 (tickers 순서대로)
weights_dict = {'GLD': 0.58194, 'QQQ': 0.41806, 'SCHD': 0.0, 'SPY': 0.0, 'TLT': 0.0}
weights = np.array([weights_dict[ticker] for ticker in tickers])

# 4. 포트폴리오 수익률, 변동성, Sharpe Ratio 계산
portfolio_return = np.dot(weights, mean_returns)
portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
sharpe_ratio = (portfolio_return - rf) / portfolio_volatility

# 5. 출력
print("🔍 주어진 비중으로 계산한 포트폴리오 성과:")
for ticker, weight in zip(tickers, weights):
    print(f"{ticker}: {weight:.2%}")

print(f"\n📈 예상 연 수익률: {portfolio_return:.2%}")
print(f"📉 예상 연 변동성: {portfolio_volatility:.2%}")
print(f"⭐ Sharpe Ratio: {sharpe_ratio:.2f}")

[*********************100%***********************]  5 of 5 completed

🔍 주어진 비중으로 계산한 포트폴리오 성과:
SPY: 0.00%
QQQ: 41.81%
SCHD: 0.00%
TLT: 0.00%
GLD: 58.19%

📈 예상 연 수익률: 7.84%
📉 예상 연 변동성: 12.99%
⭐ Sharpe Ratio: 0.45


In [22]:
import yfinance as yf
import numpy as np
import cvxpy as cp
from pypfopt import expected_returns, risk_models

# 1. 데이터 수집
tickers = ['SPY', 'QQQ', 'SCHD', 'TLT', 'GLD']
data = yf.download(tickers, start="2018-01-01", end="2024-12-31")['Close']

# 2. 기대 수익률, 공분산 행렬 계산
mu = expected_returns.mean_historical_return(data)   # 기대수익률 벡터
S = risk_models.sample_cov(data)                     # 공분산 행렬

# 3. cvxpy 변수 선언
w = cp.Variable(len(tickers))  # 각 자산 비중 변수

# 4. 목적함수 (수익률 최대화)
ret = mu.values @ w

# 5. 리스크 제약 (변동성 제한)
target_volatility = 0.15  # 15% 목표 변동성
risk = cp.quad_form(w, S.values)

# 6. 제약조건 (모든 비중 합 = 1, 비중은 음수 불가, 변동성 제한)
constraints = [
    cp.sum(w) == 1,
    w >= 0,
    risk <= target_volatility ** 2
]

# 7. 문제 정의 및 최적화
problem = cp.Problem(cp.Maximize(ret), constraints)
problem.solve()

# 8. 결과 출력
weights = w.value
weights = np.round(weights, 4)  # 소수점 4자리로 반올림

print("🔧 최적화 비중 (변동성 ≤ 15%):")
for ticker, weight in zip(tickers, weights):
    print(f"{ticker}: {weight:.4f}")

# 9. 성과 계산 (연평균 수익률, 변동성, 샤프비율)
annual_return = ret.value
annual_volatility = np.sqrt(risk.value)
risk_free_rate = 0.017  # 예: 1.7% 무위험 수익률 (변경 가능)

sharpe_ratio = (annual_return - risk_free_rate) / annual_volatility

print(f"\n📈 예상 연 수익률: {annual_return:.2%}")
print(f"📉 예상 연 변동성: {annual_volatility:.2%}")
print(f"⭐ Sharpe Ratio: {sharpe_ratio:.2f}")

load D:\Project\Python_Source\Test01\.venv\Lib\site-packages\ortools\.libs\zlib1.dll...
load D:\Project\Python_Source\Test01\.venv\Lib\site-packages\ortools\.libs\abseil_dll.dll...
load D:\Project\Python_Source\Test01\.venv\Lib\site-packages\ortools\.libs\utf8_validity.dll...
load D:\Project\Python_Source\Test01\.venv\Lib\site-packages\ortools\.libs\re2.dll...
load D:\Project\Python_Source\Test01\.venv\Lib\site-packages\ortools\.libs\libprotobuf.dll...
load D:\Project\Python_Source\Test01\.venv\Lib\site-packages\ortools\.libs\highs.dll...
load D:\Project\Python_Source\Test01\.venv\Lib\site-packages\ortools\.libs\ortools.dll...
(CVXPY) May 23 04:26:41 PM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.12.4544). Expected < 9.12.0. Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) May 23 04:26:41 PM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version

[*********************100%***********************]  5 of 5 completed

🔧 최적화 비중 (변동성 ≤ 15%):
SPY: 0.4807
QQQ: 0.5193
SCHD: 0.0000
TLT: 0.0000
GLD: 0.0000

📈 예상 연 수익률: 14.70%
📉 예상 연 변동성: 15.00%
⭐ Sharpe Ratio: 0.87


In [23]:
import yfinance as yf
import numpy as np
import cvxpy as cp
from pypfopt import expected_returns, risk_models

# 1. 데이터 수집
tickers = ['SPY', 'QQQ', 'SCHD', 'TLT', 'GLD']
data = yf.download(tickers, start="2018-01-01", end="2024-12-31")['Close']

# 2. 기대 수익률, 공분산 행렬 계산
mu = expected_returns.mean_historical_return(data)   # 기대수익률 벡터
S = risk_models.sample_cov(data)                     # 공분산 행렬

# 3. cvxpy 변수 선언
w = cp.Variable(len(tickers))  # 각 자산 비중 변수

# 4. 목적함수 (변동성 최소화)
risk = cp.quad_form(w, S.values)

# 5. 최소 목표 수익률 설정 (조정 가능)
target_return = 0.10  # 예: 연 10% 수익률 목표

# 6. 제약조건 (비중 합=1, 비중 >=0, 최소 수익률 이상)
constraints = [
    cp.sum(w) == 1,
    w >= 0,
    mu.values @ w >= target_return
]

# 7. 문제 정의 및 최적화
problem = cp.Problem(cp.Minimize(risk), constraints)
problem.solve()

# 8. 결과 출력
weights = w.value
weights = np.round(weights, 4)  # 소수점 4자리로 반올림

print(f"🔧 최적화 비중 (최소 수익률 {target_return*100:.1f}% 달성, 변동성 최소화):")
for ticker, weight in zip(tickers, weights):
    print(f"{ticker}: {weight:.4f}")

# 9. 성과 계산 (연평균 수익률, 변동성, 샤프비율)
annual_return = mu.values @ weights
annual_volatility = np.sqrt(weights.T @ S.values @ weights)
risk_free_rate = 0.017  # 예: 1.7% 무위험 수익률 (필요시 변경)

sharpe_ratio = (annual_return - risk_free_rate) / annual_volatility

print(f"\n📈 예상 연 수익률: {annual_return:.2%}")
print(f"📉 예상 연 변동성: {annual_volatility:.2%}")
print(f"⭐ Sharpe Ratio: {sharpe_ratio:.2f}")

[*********************100%***********************]  5 of 5 completed

🔧 최적화 비중 (최소 수익률 10.0% 달성, 변동성 최소화):
SPY: 0.5098
QQQ: 0.1285
SCHD: 0.0801
TLT: 0.1541
GLD: 0.1275

📈 예상 연 수익률: 10.00%
📉 예상 연 변동성: 11.22%
⭐ Sharpe Ratio: 0.74


In [24]:
import yfinance as yf
import numpy as np
import cvxpy as cp
from pypfopt import expected_returns, risk_models

# 1. 데이터 수집
tickers = ['SPY', 'QQQ', 'SCHD', 'TLT', 'GLD']
data = yf.download(tickers, start="2018-01-01", end="2024-12-31")['Close']

# 2. 기대 수익률, 공분산 행렬 계산
mu = expected_returns.mean_historical_return(data)
S = risk_models.sample_cov(data)

# 3. cvxpy 변수 선언
w = cp.Variable(len(tickers))

# 4. 목적함수 (수익률 최대화)
ret = mu.values @ w

# 5. 변동성 제약 (10% 이하)
risk = cp.quad_form(w, S.values)
target_volatility = 0.10

# 6. 제약조건 (비중 합 = 1, 비중 음수 불가, 변동성 제한)
constraints = [
    cp.sum(w) == 1,
    w >= 0,
    risk <= target_volatility ** 2
]

# 7. 문제 정의 및 최적화
problem = cp.Problem(cp.Maximize(ret), constraints)
problem.solve()

# 8. 결과 출력
weights = w.value
weights = np.round(weights, 4)

print(f"🔧 최적화 비중 (변동성 ≤ {target_volatility*100:.0f}%):")
for ticker, weight in zip(tickers, weights):
    print(f"{ticker}: {weight:.4f}")

annual_return = ret.value
annual_volatility = np.sqrt(risk.value)
risk_free_rate = 0.017

sharpe_ratio = (annual_return - risk_free_rate) / annual_volatility

print(f"\n📈 예상 연 수익률: {annual_return:.2%}")
print(f"📉 예상 연 변동성: {annual_volatility:.2%}")
print(f"⭐ Sharpe Ratio: {sharpe_ratio:.2f}")

[*********************100%***********************]  5 of 5 completed

🔧 최적화 비중 (변동성 ≤ 10%):
SPY: 0.3968
QQQ: 0.0000
SCHD: 0.1811
TLT: 0.1640
GLD: 0.2581

📈 예상 연 수익률: 7.15%
📉 예상 연 변동성: 10.00%
⭐ Sharpe Ratio: 0.55


In [25]:
import yfinance as yf
import numpy as np
import cvxpy as cp
from pypfopt import expected_returns, risk_models

tickers = ['SPY', 'QQQ', 'SCHD', 'TLT', 'GLD']
data = yf.download(tickers, start="2018-01-01", end="2024-12-31")['Close']

mu = expected_returns.mean_historical_return(data)
S = risk_models.sample_cov(data)

w = cp.Variable(len(tickers))
ret = mu.values @ w
risk = cp.quad_form(w, S.values)
target_volatility = 0.10

constraints = [
    cp.sum(w) == 1,
    w >= 0,
    risk <= target_volatility**2,
]

problem = cp.Problem(cp.Maximize(ret), constraints)
# 다른 solver 시도해보기
problem.solve(solver=cp.SCS, verbose=True)

weights = w.value
weights = np.round(weights, 4)

print(f"🔧 최적화 비중 (변동성 ≤ {target_volatility*100:.0f}%):")
for ticker, weight in zip(tickers, weights):
    print(f"{ticker}: {weight:.4f}")

annual_return = ret.value
annual_volatility = np.sqrt(risk.value)
risk_free_rate = 0.017
sharpe_ratio = (annual_return - risk_free_rate) / annual_volatility

print(f"\n📈 예상 연 수익률: {annual_return:.2%}")
print(f"📉 예상 연 변동성: {annual_volatility:.2%}")
print(f"⭐ Sharpe Ratio: {sharpe_ratio:.2f}")

[*********************100%***********************]  5 of 5 completed

                                     CVXPY                                     
                                     v1.6.5                                    
(CVXPY) May 23 04:27:20 PM: Your problem has 5 variables, 7 constraints, and 0 parameters.
(CVXPY) May 23 04:27:20 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 23 04:27:20 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 23 04:27:20 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) May 23 04:27:20 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 23 04:27:20 PM: Compiling problem (target solver=SCS).
(CVXPY

-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
(CVXPY) May 23 04:27:20 PM: Invoking solver SCS  to obtain a solution.
------------------------------------------------------------------
	       SCS v3.2.7 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 6, constraints m: 14
cones: 	  z: primal zero / dual free vars: 1
	  l: linear vars: 6
	  q: soc vars: 7, qsize: 1
settings: eps_abs: 1.0e-05, eps_rel: 1.0e-05, eps_infeas: 1.0e-07
	  alpha: 1.50, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-06
	  acceleration_lookback: 10, acceleration_interval: 10
lin-sys:  sparse-direct-amd-qdldl
	  nnz(A): 38, nnz(P): 0
--------------------------------------

In [26]:
import yfinance as yf
import numpy as np
import cvxpy as cp
from pypfopt import expected_returns, risk_models

# 1. 데이터 다운로드
tickers = ['SPY', 'QQQ', 'SCHD', 'TLT', 'GLD']
data = yf.download(tickers, start="2018-01-01", end="2024-12-31")['Close']

# 2. 기대수익률과 공분산
mu = expected_returns.mean_historical_return(data)
S = risk_models.sample_cov(data)

risk_free_rate = 0.017  # 무위험 수익률

# 3. 변동성 제한 범위 (10%~15%)
volatility_limits = np.linspace(0.10, 0.15, 11)

best_sharpe = -np.inf
best_weights = None
best_vol = None
best_return = None
best_limit = None

for target_volatility in volatility_limits:
    w = cp.Variable(len(tickers))
    ret = mu.values @ w
    risk = cp.quad_form(w, S.values)

    constraints = [
        cp.sum(w) == 1,
        w >= 0,
        risk <= target_volatility**2,
    ]

    problem = cp.Problem(cp.Maximize(ret), constraints)
    try:
        problem.solve(solver=cp.SCS, verbose=False)
    except:
        continue

    if w.value is not None:
        weights = w.value
        annual_return = ret.value
        annual_volatility = np.sqrt(risk.value)
        sharpe_ratio = (annual_return - risk_free_rate) / annual_volatility

        if sharpe_ratio > best_sharpe:
            best_sharpe = sharpe_ratio
            best_weights = weights
            best_vol = annual_volatility
            best_return = annual_return
            best_limit = target_volatility

# 결과 출력
print(f"✅ 최적 샤프비율 포트폴리오 (변동성 제한 ≤ {best_limit*100:.2f}%)")
for ticker, wgt in zip(tickers, best_weights):
    print(f"{ticker}: {wgt:.4f}")

print(f"\n- 연 수익률: {best_return:.2%}")
print(f"- 연 변동성: {best_vol:.2%}")
print(f"- Sharpe Ratio: {best_sharpe:.2f}")

[*********************100%***********************]  5 of 5 completed


✅ 최적 샤프비율 포트폴리오 (변동성 제한 ≤ 14.00%)
SPY: 0.5647
QQQ: 0.4353
SCHD: 0.0000
TLT: 0.0000
GLD: 0.0000

- 연 수익률: 13.91%
- 연 변동성: 14.00%
- Sharpe Ratio: 0.87


In [27]:
import yfinance as yf
import numpy as np
import cvxpy as cp
from pypfopt import expected_returns, risk_models

# 1. 데이터 다운로드
tickers = ['SPY', 'QQQ', 'SCHD', 'TLT', 'GLD']
data = yf.download(tickers, start="2018-01-01", end="2024-12-31")['Close']

# 2. 기대수익률과 공분산 행렬 계산
mu = expected_returns.mean_historical_return(data)
S = risk_models.sample_cov(data)

# 3. Sharpe Ratio 최적화 변수 초기화
risk_free_rate = 0.017  # 무위험 수익률
volatility_limits = np.linspace(0.05, 0.10, 11)  # 5% ~ 10%

best_sharpe = -np.inf
best_weights = None
best_vol = None
best_return = None
best_limit = None

# 4. 변동성 제한 범위 내 최적화 수행
for target_volatility in volatility_limits:
    w = cp.Variable(len(tickers))
    ret = mu.values @ w
    risk = cp.quad_form(w, S.values)

    constraints = [
        cp.sum(w) == 1,
        w >= 0,
        risk <= target_volatility**2,
    ]

    problem = cp.Problem(cp.Maximize(ret), constraints)
    try:
        problem.solve(solver=cp.SCS, verbose=False)
    except:
        continue

    if w.value is not None:
        weights = w.value
        annual_return = ret.value
        annual_volatility = np.sqrt(risk.value)
        sharpe_ratio = (annual_return - risk_free_rate) / annual_volatility

        if sharpe_ratio > best_sharpe:
            best_sharpe = sharpe_ratio
            best_weights = weights
            best_vol = annual_volatility
            best_return = annual_return
            best_limit = target_volatility

# 5. 최적 결과 출력
print(f"✅ 최적 샤프비율 포트폴리오 (변동성 제한 ≤ {best_limit*100:.2f}%)")
for ticker, wgt in zip(tickers, best_weights):
    print(f"{ticker}: {wgt:.4f}")

print(f"\n- 연 수익률: {best_return:.2%}")
print(f"- 연 변동성: {best_vol:.2%}")
print(f"- Sharpe Ratio: {best_sharpe:.2f}")

[*********************100%***********************]  5 of 5 completed


✅ 최적 샤프비율 포트폴리오 (변동성 제한 ≤ 10.00%)
SPY: 0.3968
QQQ: 0.0000
SCHD: 0.1811
TLT: 0.1640
GLD: 0.2581

- 연 수익률: 7.15%
- 연 변동성: 10.00%
- Sharpe Ratio: 0.55
